In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

import wandb

In [ ]:
# Constants
H_tower = 80
H_HR = 8
W_HR = 7



In [ ]:
# 常数计算
## DNI
def cal_DNI(H,alpha):
    """
    H: 海拔高度, km
    """
    a = 0.4237 - 0.00821 * (6 - H ) ** 2
    b = 0.5055 + 0.00595 * (6.5 - H ) ** 2
    c = 0.2711 + 0.01858 * (2.5 - H ) ** 2
    G0 = 1.366 # kW/m^2
    DNI = G0 * (a+b*np.exp(-c/np.sin(alpha)))
    return DNI

def cal_alpha():
    """
    计算太阳高度角
    """
    return None

In [ ]:
class reflect_matrix(torch.nn.Module):
    def __init__(self, num_panel, trainning_dict, initial_posit = None, initial_areas = None, initial_heights = None):
        """
        num_panel: number of panels
        parameters: the coordinates of the reflect matrix, (x, y)
        areas: the areas of each panel, can be different, paremeters: height, width - (num_panel, 2)
        heights: the heights of each panel, can be different
        """
        super(reflect_matrix, self).__init__()
        self.num_panel = num_panel
        if initial_posit:
            self.posit = torch.nn.Parameter(torch.tensor(initial_posit))
        else:
            self.posit = torch.nn.Parameter(torch.randn(num_panel, 2))
        if initial_areas:
            self.areas = torch.nn.Parameter(torch.tensor(initial_areas))
        else:
            self.areas = torch.nn.Parameter(torch.ones(num_panel))
        if initial_heights:
            self.heights = torch.nn.Parameter(torch.tensor(initial_heights))
        else:
            self.heights = torch.nn.Parameter(torch.ones(num_panel))

        for name, value in trainning_dict.items():
            if name == 'posit':
                self.posit.requires_grad = value
            elif name == 'areas':
                self.areas.requires_grad = value
            elif name == 'heights':
                self.heights.requires_grad = value
            else:
                raise ValueError('The name of the trainning_dict is not correct.')
            
        self.d_HR = torch.sqrt((self.posit[:, 0]) ** 2 + (self.posit[:, 1]) ** 2 + (H_tower - H_HR) ** 2)
        # create a list of neighbour for each panel
        self.neighbour_mask = None
        self.find_neighbour()

    def cal_efficency(self, input, lower_limit = 6*1e5):
        """
        input: the input contains the information of sun light, specifically
        - input[:, 0]: 太阳光方位角 gamma_s
        - input[:, 1]: 太阳光高度角 alpha_s
        - input[:, 2]: DNI
        """

        def cal_eta_sb():
            # TODO: calculate the eta_sb
            ## only calculate neighbour
            
            return None

        def cal_eta_cos():
            # $$cos\theta = \sqrt{\frac{1}{2}[1+\frac{1}{d_{HR}}(-xcos\alpha_ssin\gamma_s-ycos\alpha_s\cos\gamma_s+sin\alpha_s(h_1-h_2))]}$$
            eta_cos = torch.sqrt((1 + (1 / self.d_HR) * (- self.posit[:, 0] * torch.cos(input[:, 1]) * torch.sin(input[:, 0]) - \
                                self.posit[:, 1] * torch.cos(input[:, 1]) * torch.cos(input[:, 0]) + \
                                torch.sin(input[:, 1]) * (H_tower - H_HR))) / 2)
            return eta_cos
        
        def cal_eta_at():
            #calculate the eta_at
            eta_at = 0.99321 - 1.117e-4 * self.d_HR + 1.97e-8 * self.d_HR ** 2
            return eta_at
        
        def cal_eta_trunc():
            #calculate the eta_trunc
            ## beta = arctan((H_tower - H_HR) / d_HR)
            beta = torch.atan((H_tower - H_HR) / self.d_HR)
            ## cos_theta_v = \sqrt{\frac{1+cos2\theta_v}{2}}=\sqrt{\frac{1+cos(\alpha_s-\beta)}{2}}
            cos_theta_v = torch.sqrt((1 + torch.cos(2 * (input[:, 1] - beta))) / 2)
            ## cos_theta_h = =\sqrt{\frac{1+cos2\theta_h}{2}}=-\frac{1}{\sqrt{x^2+y^2}}(xsin\gamma_s+ycos\gamma_s)
            cos_theta_h = - (self.posit[:, 0] * torch.sin(input[:, 0]) + self.posit[:, 1] * \
                             torch.cos(input[:, 0]))/self.d_HR
            ## H_ratio = min(1, \frac{H_HR}{H_ref}*cos\theta_v/cos\beta)
            H_ratio = torch.min(torch.tensor(1), H_HR / (self.area[:,0] * cos_theta_v / torch.cos(beta)))
            ## W_ratio = min(1, \frac{W_HR}{W_ref}*cos\theta_h)
            W_ratio = torch.min(torch.tensor(1), W_HR / (self.area[:,1] * cos_theta_h))
            ## A_ratio = H_ratio * W_ratio
            A_ratio = H_ratio * W_ratio
            return A_ratio
        
        def cal_eta_ref():
            #calculate the eta_ref
            return 0.92
        
        
        
        # calculate eta
        eta_sb = cal_eta_sb()
        eta_cos = cal_eta_cos()
        eta_at = cal_eta_at()
        eta_trunc = cal_eta_trunc()
        eta_ref = cal_eta_ref()

        eta = eta_sb * eta_cos * eta_at * eta_trunc * eta_ref # each is a tensor of size (num_panel, )

        # calculate the efficiency per unit area
        E_field = torch.sum(input[:, 2] * eta)

        # penalty
        ## the sum of the efficiency should be larger than 0.5
        # lower_limit = lower_limit
        lower_limit_penalty = torch.min(torch.tensor(0), E_field - lower_limit)

        penalty = lower_limit_penalty*0.1
        return E_field, penalty

    def find_neighbour(self):
        """
        find the neighbour of each panel
        """
        search_range_sq = 2 * self.heights**2
        # if the distance between two panels is smaller than the search range, then they are neighbours
        # the distance between two panels
        distance_sq = (self.posit[:, 0].unsqueeze(1) - self.posit[:, 0].unsqueeze(0)) ** 2 + \
                        (self.posit[:, 1].unsqueeze(1) - self.posit[:, 1].unsqueeze(0)) ** 2
        # find the neighbour
        self.neighbour_mask = (distance_sq < search_range_sq).float()
        # the mask has no effect on the panel itself
        self.neighbour_mask[torch.arange(self.num_panel), torch.arange(self.num_panel)] = 0
        # no gradient
        self.neighbour_mask.requires_grad = False
        return None

In [ ]:
trainning_dict = {
    'posit': True,
    'areas': False,
    'heights': False
}

num_panel = 300
H_ref, W_ref = 6, 6
H_ref_base = 4

initial_posit = None
initial_areas = np.hstack((np.ones((num_panel, 1)) * H_ref, np.ones((num_panel, 1)) * W_ref))
initial_areas = torch.tensor(initial_areas)
initial_heights = torch.ones(num_panel) * H_ref_base
initial_heights = torch.tensor(initial_heights)\

input =  None
input = torch.tensor(input) # the whole input only has a size of 60

reflect_matrix = reflect_matrix(num_panel, trainning_dict, initial_posit, initial_areas, initial_heights)
E_field, penalty = reflect_matrix.cal_efficency(input)

# TODO: train the reflect_matrix
## maximize the E_field
loss = - E_field + penalty

## optimize
### set trainning hyperparameters
optimizer = torch.optim.Adam(reflect_matrix.parameters(), lr = 0.01)
num_epoch = 100
### print trainning information
print('trainning information:')
print('trainning_dict: {}'.format(trainning_dict))
for param in reflect_matrix.parameters():
    print('parameter: {}, requires_grad: {}'.format(param, param.requires_grad))
print('num_epoch: {}'.format(num_epoch))
print('optimizer: {}'.format(optimizer))


for epoch in range(num_epoch):
    optimizer.zero_grad()
    E_field, penalty = reflect_matrix.cal_efficency(input)
    loss = - E_field + penalty
    loss.backward()
    optimizer.step()
    print('epoch: {}, loss: {}, E_field: {}, penalty: {}'.format(epoch, loss, E_field, penalty))


